In [1]:
%matplotlib inline
import os, io, json, urllib, numpy as np, codecs
import glob, pandas as pd
pd.options.display.max_rows = 999

In [24]:
def h_(citation):
    citation=list(citation)
    h=0
    citation.sort(reverse=True)
    for i in range(1,len(citation)+1):
        if citation[i-1] >= i:
            h=i
    return h

In [275]:
# skip if result.txt exists
y16 = glob.glob('data/2016/*.txt')
y15 = glob.glob('data/2015/*.txt')
y14 = glob.glob('data/2014/*.txt')
y14_16=y14+y15+y16
with open("data/data.txt", "wb") as outfile:
    for f in y14_16:
        with open(f, "rb") as infile:
            lines=infile.read().split('\n')
            for line in lines:
                if len(line.split('\t'))==10:
                    outfile.write(line+'\n')

In [276]:
df=pd.read_table('data/data.txt',header=None)
df.columns=['doi','year','ref','citation','issnp','issne','journal','publisher','author','lic']
df.shape

(2995098, 10)

In [277]:
#df=df[df['author']==1]
#df.shape

In [278]:
df['year']=df['year'].astype(int)

In [279]:
df=df[df['year']<=2016][df['year']>=2014]
df.shape

C:\Users\moqri\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


(2970751, 10)

In [280]:
df['count']=1
df['cited']=df['citation'].apply(lambda x: 1 if x>0 else 0)
df['un-cited']=df['citation'].apply(lambda x: 0 if x>0 else 1)

In [281]:
df=df[(~df.issnp.isnull()) | (~df.issne.isnull())]
df.shape

(2970239, 13)

In [282]:
df=df[['year','count','citation','issnp','issne']].fillna('_')
df['issn']=df.issnp+','+df.issne
df['issn']=df.issn.str.replace(',_','')
df['issn']=df.issn.str.replace('_,','')

In [283]:
df['issn']=df.issn.apply(lambda x:x[0:9] if x[10:19]==x[0:9] else x)

In [284]:
s=df.issn.str.split(',',expand=True).stack()

In [285]:
i = s.index.get_level_values(0)

In [286]:
df2 = df.loc[i].copy()

In [287]:
df2["@id"] = s.values

In [314]:
dg=df2.groupby(['@id','year']).agg({'count':sum,'citation':sum}).reset_index()
dg['impact']=dg['citation'].div(dg['count']).round(2)
dg.shape

(57335, 5)

In [289]:
dg_h=df2[['@id','citation']].groupby(['@id']).agg(lambda x:tuple(x)).reset_index()
dg_h['h']=dg2.citation.apply(lambda x: h_(x))
dg_h.shape

(48610, 3)

In [315]:
dg_i=df2.groupby(['@id']).agg({'count':sum,'citation':sum}).reset_index()
dg_i['impact']=dg_i['citation'].div(dg_i['count']).round(2)
dg_i.shape

(48610, 4)

In [320]:
dg_flat=dg.pivot(index='@id',columns='year')
dg_flat.columns = [col[0]+'_'+str(col[1])[3] for col in dg_flat.columns.values]
dg_flat=dg_flat.reset_index()

In [325]:
dg_merge=dg_flat.merge(dg_i[['@id','count','citation','impact']])\
.merge(dg_h[['@id','h']]).fillna(0,downcast='infer')
dg_merge

,@id,count_4,count_5,count_6,citation_4,citation_5,citation_6,impact_4,impact_5,impact_6,count,citation,impact,h
0,0000-992X,1,0,0,0,0,0,0.00,0.00,0,1,0,0.00,5
1,0001-0782,298,0,0,1666,0,0,5.59,0.00,0,298,1666,5.59,5
2,0001-0898,0,12,0,0,0,0,0.00,0.00,0,12,0,0.00,9
3,0001-1452,281,4,0,1522,22,0,5.42,5.50,0,285,1544,5.42,2
4,0001-1541,376,0,0,3349,0,0,8.91,0.00,0,376,3349,8.91,5
5,0001-2092,320,0,0,223,0,0,0.70,0.00,0,320,223,0.70,3
6,0001-2785,234,15,0,15,0,0,0.06,0.00,0,249,15,0.06,1
7,0001-2815,221,0,0,807,0,0,3.65,0.00,0,221,807,3.65,3
8,0001-2998,56,0,0,350,0,0,6.25,0.00,0,56,350,6.25,6
9,0001-3072,20,0,0,67,0,0,3.35,0.00,0,20,67,3.35,3


In [329]:
dic=[]
for index, i in dg_merge[:].iterrows():
    journal={}
    journal['@id']=i['@id']
    #issn_list=[i['@id']]
    #if (i['issne2']!='') and (i['issne2']!=i['@id']):
        #issn_list.append(i['issne2'])
    #journal['e:issn']=issn_list
    journal['e:count_4']=i['count_4']
    journal['e:citation_4']=i['citation_4']
    journal['e:count_5']=i['count_5']
    journal['e:citation_5']=i['citation_5']
    journal['e:count_6']=i['count_6']
    journal['e:citation_6']=i['citation_6']
    journal['e:count']=i['count']
    journal['e:citation']=i['citation']
    journal['e:h']=i['h']
    journal['e:impact']=round(i['impact'],2)
    if journal not in dic:
        dic.append(journal)

In [330]:
with open('journals_stats.jsonld', 'w') as outfile:
    json.dump(dic, outfile)